# This is my own method to tile the pictures. 

# TODO: which format to save the Translated pictures? 

In [15]:
import pandas as pd
from osgeo import gdal
import math
import os
import glob
from PIL import Image # PIL library supports only TIF format but not newer TIFF
import numpy as np

In [11]:
data_directory = '../data/'
asf_data_directory = data_directory + '3_processed_ASF_data/'
annotated_asf_data_directory = data_directory + '6_annotated_ASF_data/'
cutted_asf_data_directory = data_directory + '6_cutted_ASF_data/'
sliced_asf_data_directory = data_directory +'7_sliced_ASF_data/'
results_directory = '../results/'

In [3]:
full_data_df = pd.read_csv(results_directory + "2022-11-19_18-44-27_full_data_df.csv", index_col=0)
full_data_df

,asf_file,image_datetime,date,minLON,minLAT,maxLON,maxLAT,widthpx,heightpx,datetime_lower,datetime_upper,ais_file
0,S1A_IW_GRDH_1SDV_20210204T015812_20210204T0158...,2021-02-04 01:58:41,2021-02-04,-118.496252,33.197789,-117.964359,33.855266,5921,7319,2021-02-04 01:48:41,2021-02-04 02:08:41,AIS_2021_02_04.csv
1,S1B_IW_GRDH_1SDV_20210105T015737_20210105T0158...,2021-01-05 01:58:06,2021-01-05,-118.575269,33.251929,-117.946359,33.930247,7001,7551,2021-01-05 01:48:06,2021-01-05 02:08:06,AIS_2021_01_05.csv
2,S1B_IW_GRDH_1SDV_20210306T015735_20210306T0158...,2021-03-06 01:58:04,2021-03-06,-118.482835,33.199482,-117.980677,33.829740,5590,7016,2021-03-06 01:48:04,2021-03-06 02:08:04,AIS_2021_03_06.csv


# GDAL_TRANSLATE documentation
    - srcwin <xoff> <yoff> <xsize> <ysize>
    window = (offset_x, offset_y, size_x, size_y)
    gdal.Translate('output_crop_raster.tif', 'input_raster.tif', srcWin = window)
    
- http://erouault.blogspot.com/2015/10/gdal-and-ogr-utilities-as-library.html
- https://gdal.org/programs/gdal_translate.html
- https://stackoverflow.com/questions/48531082/cropping-a-raster-file-using-gdal-w-python
- https://gis.stackexchange.com/questions/42584/how-to-call-gdal-translate-from-python-code

In [6]:
CHOSEN_PICTURE_SIZE = 500 # pictures of example size 1000x1000 will be cutted

for index, row in full_data_df.iterrows():
    print(f"File: {index}")
    file_name = row['asf_file']
    print(f"File name: {file_name}")
    in_ds = gdal.Open(asf_data_directory + file_name)
    image_height = row['heightpx']
    image_width = row['widthpx']
    print(f"Image_size: {image_width}x{image_height} px")
    image_height = row['heightpx']
    nr_x_edges = math.ceil(image_width / CHOSEN_PICTURE_SIZE)
    nr_y_edges = math.ceil(image_height / CHOSEN_PICTURE_SIZE)
    print(f"X_edges: {nr_x_edges}, Y_edges: {nr_y_edges}")
    print(f"All_edges = x_edges*y_edges = {nr_x_edges*nr_y_edges}")
    # cut the squares
    subset_number = 0
    for i in range(nr_x_edges):
        for j in range(nr_y_edges):
            #print(i, j)
            subset_number += 1
            window = [i*CHOSEN_PICTURE_SIZE, j*CHOSEN_PICTURE_SIZE, CHOSEN_PICTURE_SIZE, CHOSEN_PICTURE_SIZE]
            out_ds = gdal.Translate(cutted_asf_data_directory + file_name + f'_subset{subset_number}.tif', in_ds, srcWin=window)
            out_width = out_ds.RasterXSize
            out_height = out_ds.RasterYSize
            print(f"Out picture {subset_number} width and height: {out_width}x{out_height} px")
    break

File: 0
File name: S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.tif
Image_size: 5921x7319 px
X_edges: 12, Y_edges: 15
All_edges = x_edges*y_edges = 180
Out picture 1 width and height: 500x500 px
Out picture 2 width and height: 500x500 px
Out picture 3 width and height: 500x500 px
Out picture 4 width and height: 500x500 px
Out picture 5 width and height: 500x500 px
Out picture 6 width and height: 500x500 px
Out picture 7 width and height: 500x500 px
Out picture 8 width and height: 500x500 px
Out picture 9 width and height: 500x500 px
Out picture 10 width and height: 500x500 px
Out picture 11 width and height: 500x500 px
Out picture 12 width and height: 500x500 px
Out picture 13 width and height: 500x500 px
Out picture 14 width and height: 500x500 px
Out picture 15 width and height: 500x500 px
Out picture 16 width and height: 500x500 px
Out picture 17 width and height: 500x500 px
Out picture 18 width and height: 500x500 px
Out picture 19 width and height: 500x50

# Sample script
Image object slicer: 
https://github.com/natanjunges/image-object-slicer

https://towardsdatascience.com/tile-slice-yolo-dataset-for-small-objects-detection-a75bf26f7fa2

 This file is part of image-object-slicer
    # Copyright (C) 2022  Natan Junges <natanajunges@gmail.com>
    #
    # image-object-slicer is free software: you can redistribute it and/or modify
    # it under the terms of the GNU General Public License as published by
    # the Free Software Foundation, either version 3 of the License, or
    # any later version.
    #
    # image-object-slicer is distributed in the hope that it will be useful,
    # but WITHOUT ANY WARRANTY; without even the implied warranty of
    # MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    # GNU General Public License for more details.
    #
    # You should have received a copy of the GNU General Public License
    # along with image-object-slicer.  If not, see <https://www.gnu.org/licenses/>.

https://github.com/slanj/yolo-tiling/blob/main/tile_yolo.py

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from shapely.geometry import Polygon
import glob
import argparse
import os
import random
from shutil import copyfile
 
def tiler(imnames, newpath, falsepath, slice_size, ext):
    for imname in imnames:
        im = Image.open(imname)
        imr = np.array(im, dtype=np.uint8)
        height = imr.shape[0]
        width = imr.shape[1]
        labname = imname.replace(ext, '.txt')
        labels = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])
        
        # we need to rescale coordinates from 0-1 to real image height and width
        labels[['x1', 'w']] = labels[['x1', 'w']] * width
        labels[['y1', 'h']] = labels[['y1', 'h']] * height
        
        boxes = []
        
        # convert bounding boxes to shapely polygons. We need to invert Y and find polygon vertices from center points
        for row in labels.iterrows():
            x1 = row[1]['x1'] - row[1]['w']/2
            y1 = (height - row[1]['y1']) - row[1]['h']/2
            x2 = row[1]['x1'] + row[1]['w']/2
            y2 = (height - row[1]['y1']) + row[1]['h']/2

            boxes.append((int(row[1]['class']), Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])))
        
        counter = 0
        print('Image:', imname)
        # create tiles and find intersection with bounding boxes for each tile
        for i in range((height // slice_size)):
            for j in range((width // slice_size)):
                x1 = j*slice_size
                y1 = height - (i*slice_size)
                x2 = ((j+1)*slice_size) - 1
                y2 = (height - (i+1)*slice_size) + 1

                pol = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
                imsaved = False
                slice_labels = []

                for box in boxes:
                    if pol.intersects(box[1]):
                        inter = pol.intersection(box[1])        
                        
                        if not imsaved:
                            sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                            sliced_im = Image.fromarray(sliced)
                            filename = imname.split('/')[-1]
                            slice_path = newpath + "/" + filename.replace(ext, f'_{i}_{j}{ext}')                            
                            slice_labels_path = newpath + "/" + filename.replace(ext, f'_{i}_{j}.txt')                            
                            print(slice_path)
                            sliced_im.save(slice_path)
                            imsaved = True                    
                        
                        # get smallest rectangular polygon (with sides parallel to the coordinate axes) that contains the intersection
                        new_box = inter.envelope 
                        
                        # get central point for the new bounding box 
                        centre = new_box.centroid
                        
                        # get coordinates of polygon vertices
                        x, y = new_box.exterior.coords.xy
                        
                        # get bounding box width and height normalized to slice size
                        new_width = (max(x) - min(x)) / slice_size
                        new_height = (max(y) - min(y)) / slice_size
                        
                        # we have to normalize central x and invert y for yolo format
                        new_x = (centre.coords.xy[0][0] - x1) / slice_size
                        new_y = (y1 - centre.coords.xy[1][0]) / slice_size
                        
                        counter += 1

                        slice_labels.append([box[0], new_x, new_y, new_width, new_height])
                
                if len(slice_labels) > 0:
                    slice_df = pd.DataFrame(slice_labels, columns=['class', 'x1', 'y1', 'w', 'h'])
                    print(slice_df)
                    slice_df.to_csv(slice_labels_path, sep=' ', index=False, header=False, float_format='%.6f')
                
                if not imsaved and falsepath:
                    sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                    sliced_im = Image.fromarray(sliced)
                    filename = imname.split('/')[-1]
                    slice_path = falsepath + "/" + filename.replace(ext, f'_{i}_{j}{ext}')                

                    sliced_im.save(slice_path)
                    print('Slice without boxes saved')
                    imsaved = True

def splitter(target, target_upfolder, ext, ratio):
    imnames = glob.glob(f'{target}/*{ext}')
    names = [name.split('/')[-1] for name in imnames]

    # split dataset for train and test

    train = []
    test = []
    for name in names:
        if random.random() > ratio:
            test.append(os.path.join(target, name))
        else:
            train.append(os.path.join(target, name))
    print('train:', len(train))
    print('test:', len(test))

    # we will put test.txt, train.txt in a folder one level higher than images

    # save train part
    with open(f'{target_upfolder}/train.txt', 'w') as f:
        for item in train:
            f.write("%s\n" % item)

    # save test part
    with open(f'{target_upfolder}/test.txt', 'w') as f:
        for item in test:
            f.write("%s\n" % item)

if __name__ == "__main__":
    # Initialize parser
    parser = argparse.ArgumentParser()

    parser.add_argument("-source", default="./yolosample/ts/", help = "Source folder with images and labels needed to be tiled")
    parser.add_argument("-target", default="./yolosliced/ts/", help = "Target folder for a new sliced dataset")
    parser.add_argument("-ext", default=".JPG", help = "Image extension in a dataset. Default: .JPG")
    parser.add_argument("-falsefolder", default=None, help = "Folder for tiles without bounding boxes")
    parser.add_argument("-size", type=int, default=416, help = "Size of a tile. Dafault: 416")
    parser.add_argument("-ratio", type=float, default=0.8, help = "Train/test split ratio. Dafault: 0.8")

    args = parser.parse_args()

    imnames = glob.glob(f'{args.source}/*{args.ext}')
    labnames = glob.glob(f'{args.source}/*.txt')
    
    if len(imnames) == 0:
        raise Exception("Source folder should contain some images")
    elif len(imnames) != len(labnames):
        raise Exception("Dataset should contain equal number of images and txt files with labels")

    if not os.path.exists(args.target):
        os.makedirs(args.target)
    elif len(os.listdir(args.target)) > 0:
        raise Exception("Target folder should be empty")

    # classes.names should be located one level higher than images   
    # this file is not changing, so we will just copy it to a target folder 
    upfolder = os.path.join(args.source, '..' )
    target_upfolder = os.path.join(args.target, '..' )
    if not os.path.exists(os.path.join(upfolder, 'classes.names')):
        print('classes.names not found. It should be located one level higher than images')
    else:
        copyfile(os.path.join(upfolder, 'classes.names'), os.path.join(target_upfolder, 'classes.names'))
    
    if args.falsefolder:
        if not os.path.exists(args.falsefolder):
            os.makedirs(args.falsefolder)
        elif len(os.listdir(args.falsefolder)) > 0:
            raise Exception("Folder for tiles without boxes should be empty")

    tiler(imnames, args.target, args.falsefolder, args.size, args.ext)
    splitter(args.target, target_upfolder, args.ext, args.ratio)

### Second version

In [29]:
from csv import DictReader

In [44]:
annotations = annotated_asf_data_directory+'S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.txt'
annotation_format = 'yolo'

In [42]:
glob = "obj_*_data/**/*.txt"
labels =  "obj.names"

def parse_labels(cls, file):
    with open(file) as fp:
        labels = fp.readlines()

    return [label.strip() for label in labels]

def parse_file(cls, file, labels_list):
    """Parse a YOLO annotation file to a usable dict format."""
    with open(file, newline="") as fp:
        data = DictReader(fp, ["label_id", "cx", "cy", "rw", "rh"], delimiter=" ")
        name = ".".join(file.split("/")[-1].split(".")[:-1])
        slices = []
        labels = set()

        for obj in data:
            object_label = labels_list[int(obj.get("label_id"))]
            labels.add(object_label)
            cx = float(obj.get("cx"))
            cy = float(obj.get("cy"))
            rw = float(obj.get("rw"))
            rh = float(obj.get("rh"))
            slices.append({
                "xmin": cx - (rw/2),
                "ymin": cy - (rh/2),
                "xmax": cx + (rw/2),
                "ymax": cy + (rh/2),
                "label": object_label
            })

        return {"name": name, "slices": slices, "labels": labels}
    
def find_annotation_files(format, path):
    """Find all annotation files from a specific path."""
    print("Finding annotation files: ", end="")
    files = list(pathlib.Path(path).glob(format.glob))

    if len(files) > 0:
        files = [str(file) for file in files]

    if issubclass(format, SingleFileAnnotationParser) and len(files) > 1:
        raise Exception("Could not find a unique annotation file: {}".format(files))

    print("{0}/{0}".format(len(files)))

    if len(files) > 0 and format.labels is not None:
        print("Finding labels file")
        labels = list(pathlib.Path(path).glob(format.labels))

        if len(labels) == 0:
            raise Exception("Could not find labels file: {}".format(format.labels))
        else:
            labels = [str(label) for label in labels]

        if len(labels) > 1:
            raise Exception("Could not find a unique labels file: {}".format(labels))

        return (files, labels[0])
    else:
        return (files, None)

In [45]:
annotation_files = find_annotation_files(annotation_format, annotations)
if len(annotation_files[0]) > 0:
    parsed_annotation_files = parse_annotation_files(formats.get(args.format), annotation_files, args.workers)

    if len(parsed_annotation_files) > 0:
        make_dir(args.save)
        create_label_dirs(parsed_annotation_files.get("labels"), args.save)
        slice_images(args.images, parsed_annotation_files.get("names"), parsed_annotation_files.get("slice_groups"), args.padding, args.save, args.workers)
    else:
        print("Found no slices")
else:
    print("Found no annotation file")

Finding annotation files: 

AttributeError: 'str' object has no attribute 'glob'

In [32]:
import argparse
import os
from tqdm import tqdm
from PIL import Image
from multiprocessing import Pool, cpu_count
import pathlib

# from .SingleFileAnnotationParser import SingleFileAnnotationParser
# from .PascalVOCParser import PascalVOCParser
# from .COCOParser import COCOParser
# from .CVATImagesParser import CVATImagesParser
# from .DatumaroParser import DatumaroParser
# from .KITTIParser import KITTIParser
# from .LabelMeParser import LabelMeParser
# from .OpenImagesParser import OpenImagesParser
# from .WIDERFaceParser import WIDERFaceParser
# from .YOLOParser import YOLOParser

__version__ = "1.12.3"

formats = {
    # The first is always the default
    "pascalvoc": PascalVOCParser,
    "coco": COCOParser,
    "cvatimages": CVATImagesParser,
    "datumaro": DatumaroParser,
    "kitti": KITTIParser,
    "labelme": LabelMeParser,
    "openimages": OpenImagesParser,
    "widerface": WIDERFaceParser,
    "yolo": YOLOParser
}

def main():
    parser = argparse.ArgumentParser(description="Slice objects from images using annotation files")
    parser.add_argument("-v", "--version", action="version", version="%(prog)s " + __version__)
    parser.add_argument("annotations", help="A path to the directory with the annotation files")
    parser.add_argument("images", help="A path to the directory with the input images")
    parser.add_argument("save", help="A path to the directory to save the image slices to")
    format_choices = list(formats.keys())
    parser.add_argument("-f", "--format", choices=format_choices, default=format_choices[0], help="The format of the annotation files (default is {})".format(format_choices[0]))
    parser.add_argument("-p", "--padding", type=int, default=0, help="The amount of padding (in pixels) to add to each image slice")
    parser.add_argument("-w", "--workers", type=int, default=cpu_count(), help="The number of parallel workers to run (default is cpu count)")
    args = parser.parse_args()
    annotation_files = find_annotation_files(formats.get(args.format), args.annotations)

    if len(annotation_files[0]) > 0:
        parsed_annotation_files = parse_annotation_files(formats.get(args.format), annotation_files, args.workers)

        if len(parsed_annotation_files) > 0:
            make_dir(args.save)
            create_label_dirs(parsed_annotation_files.get("labels"), args.save)
            slice_images(args.images, parsed_annotation_files.get("names"), parsed_annotation_files.get("slice_groups"), args.padding, args.save, args.workers)
        else:
            print("Found no slices")
    else:
        print("Found no annotation file")

def find_annotation_files(format, path):
    """Find all annotation files from a specific path."""
    print("Finding annotation files: ", end="")
    files = list(pathlib.Path(path).glob(format.glob))

    if len(files) > 0:
        files = [str(file) for file in files]

    if issubclass(format, SingleFileAnnotationParser) and len(files) > 1:
        raise Exception("Could not find a unique annotation file: {}".format(files))

    print("{0}/{0}".format(len(files)))

    if len(files) > 0 and format.labels is not None:
        print("Finding labels file")
        labels = list(pathlib.Path(path).glob(format.labels))

        if len(labels) == 0:
            raise Exception("Could not find labels file: {}".format(format.labels))
        else:
            labels = [str(label) for label in labels]

        if len(labels) > 1:
            raise Exception("Could not find a unique labels file: {}".format(labels))

        return (files, labels[0])
    else:
        return (files, None)

def parse_annotation_file(args):
    """Parse a specific annotation file to a usable dict format."""
    format = args[0]
    file = args[1]
    labels = args[2]

    try:
        parse = format.parse_file(file, labels)
        # Sort left-to-right, top-to-bottom
        parse.get("slices").sort(key=lambda slice: (slice.get("xmin"), slice.get("ymin"), slice.get("xmax"), slice.get("ymax")))
        return parse
    except Exception as e:
        # Just error if a single file cannot be read
        print("Error parsing annotation file: " + str(e))

def parse_annotation_item(args):
    """Parse a specific annotation item to a usable dict format."""
    format = args[0]
    item = args[1]

    try:
        parse = format.parse_item(item)
        # Sort left-to-right, top-to-bottom
        parse.get("slices").sort(key=lambda slice: (slice.get("xmin"), slice.get("ymin"), slice.get("xmax"), slice.get("ymax")))
        return parse
    except Exception as e:
        # Just error if a single item cannot be read
        print("Error parsing annotation item: " + str(e))

def parse_annotation_files(format, files, workers):
    """Parse all annotation files."""
    names = []
    slice_groups = []
    labels = set()
    labels_list = None

    if files[1] is not None:
        labels_list = format.parse_labels(files[1])

    if issubclass(format, SingleFileAnnotationParser):
        try:
            split = format.split_file(files[0][0], labels_list)
        except Exception as e:
            # Raise because this is the only file
            print("Error parsing annotation file:")
            raise e

        with Pool(workers) as pool:
            for parses in tqdm(pool.imap_unordered(parse_annotation_item, ((format, item) for item in split)), desc="Parsing annotation file"):
                if parses is not None:
                    labels = labels.union(parses.get("labels"))
                    names.append(parses.get("name"))
                    slice_groups.append(parses.get("slices"))
    else:
        with Pool(workers) as pool:
            for parses in tqdm(pool.imap_unordered(parse_annotation_file, ((format, file, labels_list) for file in files[0])), desc="Parsing annotation files", total=len(files[0])):
                if parses is not None and len(parses.get("slices")) > 0:
                    labels = labels.union(parses.get("labels"))
                    names.append(parses.get("name"))
                    slice_groups.append(parses.get("slices"))

    return {"names": names, "slice_groups": slice_groups, "labels": labels}

def slice_images(images_path, names, slice_groups, padding, save_path, workers):
    """Loop through all slice groups and slice each image."""
    with Pool(workers) as pool:
        for _ in tqdm(pool.imap_unordered(slice_image, ((images_path, name, slices, padding, save_path) for name, slices in zip(names, slice_groups))), desc="Slicing images", total=len(slice_groups)):
            pass

def slice_image(args):
    """Slice an image from slices."""
    images_path = args[0]
    name = args[1].split(".")

    if len(name) == 1:
        name = name[0]
        files = list(pathlib.Path(images_path).glob(name + ".*"))

        if len(files) > 0:
            files = [file.name for file in files]

        if len(files) == 0:
            print("No file candidate found: {}.*".format(name))
            return
        elif len(files) == 1:
            name = files[0].split(".")
        else:
            print("Multiple file candidates found: {}".format(files))
            return

    extension = name[-1]
    name = ".".join(name[:-1])

    slices = args[2]
    padding = args[3]
    save_path = args[4]
    image = Image.open(os.path.join(images_path, "{}.{}".format(name, extension)))

    for i, slice in enumerate(slices):
        # Floating values for the coordinates are relative to the image size

        if type(slice.get("xmin")) is float:
            slice["xmin"] = round(slice.get("xmin") * image.width)

        if type(slice.get("ymin")) is float:
            slice["ymin"] = round(slice.get("ymin") * image.height)

        if type(slice.get("xmax")) is float:
            slice["xmax"] = round(slice.get("xmax") * image.width)

        if type(slice.get("ymax")) is float:
            slice["ymax"] = round(slice.get("ymax") * image.height)

        # Create the bounding box to slice from
        bndbox = (max(0, slice.get("xmin") - padding), max(0, slice.get("ymin") - padding), min(slice.get("xmax") + padding, image.width), min(slice.get("ymax") + padding, image.height))
        image_slice = image.crop(bndbox)

        try:
            image_slice.save(os.path.join(save_path, slice.get("label"), "{}-{}-{}.{}".format(name, slice.get("label"), i, extension)))
        except Exception as  e:
            # Just error if a single image does not save
            print("Error saving image slice: " + str(e))

def create_label_dirs(labels, save_path):
    """Create all label directories."""
    for label in tqdm(labels, desc="Creating directories"):
        make_dir(save_path, label)

def make_dir(path, name=""):
    """Create a directory if it does not already exist."""
    path = os.path.abspath(os.path.join(path, name))

    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except Exception as e:
            # Raise if directory cannot be made, because image slices will not be saved
            print("Error creating directory:")
            raise e

ImportError: attempted relative import with no known parent package

#### Second method

In [5]:
os.listdir(annotated_asf_data_directory)

['.gitkeep',
 'S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png',
 'S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png.aux.xml',
 'S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.tif',
 'S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.txt']

In [10]:
glob.glob(annotated_asf_data_directory + '*.PNG')

['../data/6_annotated_ASF_data\\S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png']

In [28]:
with open('../data/6_annotated_ASF_data\\S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.txt','a',errors='ignore') as f:
    contents = f.read()
contents = contents.rstrip("\n").decode("utf-16")
contents = contents.split("\r\n")

UnsupportedOperation: not readable

In [24]:
labels = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'], encoding='utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [46]:
labname

'../data/6_annotated_ASF_data\\S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png'

In [59]:
with open(labname, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()

In [60]:
lines

['PNG\n',
 '\x1a\n',
 '\x00\x00\x00\n',
 'IHDR\x00\x00\x17!\x00\x00\x1c\x08\x04\x00\x00\x00\x16~\x00\x00 \x00IDATxݻc\x11Q$\x02\x16QdJ\x0f\n',
 '\n',
 '\x0fT"m\x02ٙ\x1dt9R\n',
 'v|\\\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [53]:
with open(labname, 'rb') as f:
    contents = f.read()

<_io.BufferedReader name='../data/6_annotated_ASF_data\\S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png'>


In [54]:
pd.DataFrame(contents)

ValueError: DataFrame constructor not properly called!

In [50]:
labels = pd.read_csv(labname, sep=" ", names=['class', 'x1', 'y1', 'w', 'h'])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [69]:
annotated_asf_data_directory

'../data/6_annotated_ASF_data/'

In [68]:
glob.glob(annotated_asf_data_directory + '/*.PNG')

AttributeError: 'str' object has no attribute 'glob'

In [75]:
imname

'../data/6_annotated_ASF_data/S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png'

In [76]:
imname.replace('png', 'csv')

'../data/6_annotated_ASF_data/S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.csv'

In [71]:
labname

'../data/6_annotated_ASF_data/S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png'

In [73]:
pd.read_csv('../data/6_annotated_ASF_data/S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.csv')

,class,x1,y1,w,h
0,0,0.408208,0.204263,0.008445,0.006832
1,0,0.681642,0.324908,0.008445,0.006832
2,0,0.803749,0.201394,0.008445,0.006832
3,0,0.724033,0.155076,0.008445,0.006832
4,0,0.182233,0.010111,0.008445,0.006832
...,...,...,...,...,...
269,0,0.522547,0.143462,0.008445,0.006832
270,0,0.541631,0.227900,0.008445,0.006832
271,0,0.554974,0.170515,0.008445,0.006832
272,0,0.792940,0.195519,0.008445,0.006832


In [84]:
import pandas as pd
import numpy as np
from PIL import Image
from shapely.geometry import Polygon
import glob
import argparse
import os
import random
from shutil import copyfile

img_input_format = '.png'
# get all image names
#imnames = glob.glob(annotated_asf_data_directory + '*.PNG')
imnames = ['../data/6_annotated_ASF_data/S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png']

# specify path for a new tiled dataset
newpath = sliced_asf_data_directory + '/ts/' #'sampletiled/ts/'
falsepath = sliced_asf_data_directory + '/false/' #'sampletiled/false/'

# specify slice width=height
slice_size = 500 #416

# tile all images in a loop

for imname in imnames:
    im = Image.open(imname)
    imr = np.array(im, dtype=np.uint8)
    height = imr.shape[0]
    width = imr.shape[1]
    #labname = imname.replace('.png', '.txt')
    #labels = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])
    labname = imname.replace(img_input_format, '.csv')
    labels = pd.read_csv(labname)
    
    # we need to rescale coordinates from 0-1 to real image height and width
    labels[['x1', 'w']] = labels[['x1', 'w']] * width
    labels[['y1', 'h']] = labels[['y1', 'h']] * height
    
    boxes = []
    
    # convert bounding boxes to shapely polygons. We need to invert Y and find polygon vertices from center points
    for row in labels.iterrows():
        x1 = row[1]['x1'] - row[1]['w']/2
        y1 = (height - row[1]['y1']) - row[1]['h']/2
        x2 = row[1]['x1'] + row[1]['w']/2
        y2 = (height - row[1]['y1']) + row[1]['h']/2

        boxes.append((int(row[1]['class']), Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])))
    
    counter = 0
    print('Image:', imname)
    # create tiles and find intersection with bounding boxes for each tile
    for i in range((height // slice_size)):
        for j in range((width // slice_size)):
            x1 = j*slice_size
            y1 = height - (i*slice_size)
            x2 = ((j+1)*slice_size) - 1
            y2 = (height - (i+1)*slice_size) + 1

            pol = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
            imsaved = False
            slice_labels = []

            for box in boxes:
                if pol.intersects(box[1]):
                    inter = pol.intersection(box[1])        
                    
                    if not imsaved:
                        sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                        sliced_im = Image.fromarray(sliced)
                        filename = imname.split('/')[-1]
                        slice_path = newpath + filename.replace(img_input_format, f'_{i}_{j}{img_input_format}')
                        
                        slice_labels_path = newpath + filename.replace(img_input_format, f'_{i}_{j}.txt')
                        
                        print(slice_path)
                        sliced_im.save(slice_path)
                        imsaved = True                    
                    
                    # get the smallest polygon (with sides parallel to the coordinate axes) that contains the intersection
                    new_box = inter.envelope 
                    
                    # get central point for the new bounding box 
                    centre = new_box.centroid
                    
                    # get coordinates of polygon vertices
                    x, y = new_box.exterior.coords.xy
                    
                    # get bounding box width and height normalized to slice size
                    new_width = (max(x) - min(x)) / slice_size
                    new_height = (max(y) - min(y)) / slice_size
                    
                    # we have to normalize central x and invert y for yolo format
                    new_x = (centre.coords.xy[0][0] - x1) / slice_size
                    new_y = (y1 - centre.coords.xy[1][0]) / slice_size
                    
                    counter += 1

                    slice_labels.append([box[0], new_x, new_y, new_width, new_height])
            
            # save txt with labels for the current tile
            if len(slice_labels) > 0:
                slice_df = pd.DataFrame(slice_labels, columns=['class', 'x1', 'y1', 'w', 'h'])
                print(slice_df)
                slice_df.to_csv(slice_labels_path, sep=' ', index=False, header=False, float_format='%.6f')
            
            # if there are no bounding boxes intersect current tile, save this tile to a separate folder 
            if not imsaved:
                sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                sliced_im = Image.fromarray(sliced)
                filename = imname.split('/')[2]
                slice_path = falsepath + filename.replace(img_input_format, f'_{i}_{j}{img_input_format}')                

                sliced_im.save(slice_path)
                print('Slice without boxes saved')
                imsaved = True

Image: ../data/6_annotated_ASF_data/S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png


ValueError: unknown file extension: 

In [52]:
df = pd.read_fwf(labname)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [85]:
def tiler(imnames, newpath, falsepath, slice_size, ext):
    for imname in imnames:
        im = Image.open(imname)
        imr = np.array(im, dtype=np.uint8)
        height = imr.shape[0]
        width = imr.shape[1]
        labname = imname.replace(ext, '.txt')
        labels = pd.read_csv(labname, sep=' ', names=['class', 'x1', 'y1', 'w', 'h'])
        
        # we need to rescale coordinates from 0-1 to real image height and width
        labels[['x1', 'w']] = labels[['x1', 'w']] * width
        labels[['y1', 'h']] = labels[['y1', 'h']] * height
        
        boxes = []
        
        # convert bounding boxes to shapely polygons. We need to invert Y and find polygon vertices from center points
        for row in labels.iterrows():
            x1 = row[1]['x1'] - row[1]['w']/2
            y1 = (height - row[1]['y1']) - row[1]['h']/2
            x2 = row[1]['x1'] + row[1]['w']/2
            y2 = (height - row[1]['y1']) + row[1]['h']/2

            boxes.append((int(row[1]['class']), Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])))
        
        counter = 0
        print('Image:', imname)
        # create tiles and find intersection with bounding boxes for each tile
        for i in range((height // slice_size)):
            for j in range((width // slice_size)):
                x1 = j*slice_size
                y1 = height - (i*slice_size)
                x2 = ((j+1)*slice_size) - 1
                y2 = (height - (i+1)*slice_size) + 1

                pol = Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)])
                imsaved = False
                slice_labels = []

                for box in boxes:
                    if pol.intersects(box[1]):
                        inter = pol.intersection(box[1])        
                        
                        if not imsaved:
                            sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                            sliced_im = Image.fromarray(sliced)
                            filename = imname.split('/')[-1]
                            slice_path = newpath + "/" + filename.replace(ext, f'_{i}_{j}{ext}')                            
                            slice_labels_path = newpath + "/" + filename.replace(ext, f'_{i}_{j}.txt')                            
                            print(slice_path)
                            sliced_im.save(slice_path)
                            imsaved = True                    
                        
                        # get smallest rectangular polygon (with sides parallel to the coordinate axes) that contains the intersection
                        new_box = inter.envelope 
                        
                        # get central point for the new bounding box 
                        centre = new_box.centroid
                        
                        # get coordinates of polygon vertices
                        x, y = new_box.exterior.coords.xy
                        
                        # get bounding box width and height normalized to slice size
                        new_width = (max(x) - min(x)) / slice_size
                        new_height = (max(y) - min(y)) / slice_size
                        
                        # we have to normalize central x and invert y for yolo format
                        new_x = (centre.coords.xy[0][0] - x1) / slice_size
                        new_y = (y1 - centre.coords.xy[1][0]) / slice_size
                        
                        counter += 1

                        slice_labels.append([box[0], new_x, new_y, new_width, new_height])
                
                if len(slice_labels) > 0:
                    slice_df = pd.DataFrame(slice_labels, columns=['class', 'x1', 'y1', 'w', 'h'])
                    print(slice_df)
                    slice_df.to_csv(slice_labels_path, sep=' ', index=False, header=False, float_format='%.6f')
                
                if not imsaved and falsepath:
                    sliced = imr[i*slice_size:(i+1)*slice_size, j*slice_size:(j+1)*slice_size]
                    sliced_im = Image.fromarray(sliced)
                    filename = imname.split('/')[-1]
                    slice_path = falsepath + "/" + filename.replace(ext, f'_{i}_{j}{ext}')                

                    sliced_im.save(slice_path)
                    print('Slice without boxes saved')
                    imsaved = True

In [ ]:
tiler([imnames='../data/6_annotated_ASF_data/S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC.png'],
     newpath=,
    falsepath=, 
    slice_size=, 
    ext=)

# OLD

In [12]:
# im = Image.open(asf_data_directory + "S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC.tif")
# imgwidth = im.size[0]
# imgheight = im.size[1]

UnidentifiedImageError: cannot identify image file '../data/3_processed_ASF_data/S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC.tif'

In [5]:
# ds = gdal.Open(asf_data_directory + "S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC.tif")
# ds = gdal.Translate(cutted_asf_data_directory + 'S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC_subset1.tif', ds, srcWin=[0, 0, 1, 1])
# print(gdal.Info(ds, deserialize=True))

In [38]:
math.ceil(12.01)

13